# Building a Spam Filter with Naive Bayes
In this project the first task is to teach the computer on how to classify sms messages as spam or non-spam. Multinomial Naives Bayes algorithm will be used along with a dataset containing 5,572 messages that are already classified by humans. Overall goal is to build a spam filter with an accuracy of 80% or more.

The Sms Spam Collection Data Set can be downloaded from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection)

More details on the data can be accessed on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition)


# Exploring the DataSet

In [1]:
import pandas as pd

# Read in the dataset
sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print('Rows:', sms_spam.shape[0], '\nColumns:', sms_spam.shape[1])

# Percentage of spam and non spam messages
sms_spam['Label'].value_counts(normalize=True) *100

Rows: 5572 
Columns: 2


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

In [2]:
sms_spam.head(5)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Overall, the dataset has a total of 5,572 rows and 2 columns. Around 86.6% messages are classified as ham (non-spam) and 13.4% as spam.

# Training and Test Set
Now it's time to build the spam filter. Before creating it, it would be very helpful to first think of a way of testing how well it works.

When creating a software, a good rule of thumb is that designing the test comes before creating the software. If a software is written first, then it's tempting to come up with biased test just to make sure the software passes it.

Once the sam filter is done, the dataset will be split in two categories:
- A training set, to train the computer on how to classify messages.
    - The training set will have 4,458 messages (about 80% of the dataset)
- A test set, to test how good the filter is with classifying new messages
    - The test will have 1,114 messages (about 20% of the dataset)

In [3]:
randomized_sms_spam = sms_spam.copy().sample(frac=1, random_state=1)
randomized_sms_spam.head(5)

,Label,SMS
1078,ham,"Yep, by the pretty sculpture"
4028,ham,"Yes, princess. Are you going to make me moan?"
958,ham,Welp apparently he retired
4642,ham,Havent.
4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [4]:
# creating training set
training_set = randomized_sms_spam[:4458]
training_set.shape

# Calculate index for split
training_set_index = round(len(randomized_sms_spam) * 0.8)

# creating training/test sets
training_set = randomized_sms_spam[:training_set_index].reset_index(drop=True)
test_set = randomized_sms_spam[training_set_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [5]:
# percentage of spam/ham in training set
training_set['Label'].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [6]:
# percentage of spam/ham in test set
test_set['Label'].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

Percentages of ham and spam between training, test, and full dataset are fairly similar in comparison.

# Data Cleaning
To calculate probabilities needed by the algorithm, the data needs to be cleaned in a format that will allow us to easily extract the information needed.

We'll begin by removing punctuation, then lowercase all the words.

Then, transform the data to a new format:

<img src="https://camo.githubusercontent.com/27a4a0a699bd8f0713d73347abe2929c267a03d5/68747470733a2f2f64712d636f6e74656e742e73332e616d617a6f6e6177732e636f6d2f3433332f637067705f646174617365745f332e706e67" alt="img" data-canonical-src="https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png">


# Letter Case and Punctuation

In [7]:
# Before cleaning
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [8]:
# After cleaning
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


# Creating the Vocabulary
With the exception of the `Label` column, every other column will be transformed

In [9]:
# transform each message into a list of words
training_set['SMS'] = training_set['SMS'].str.split()

# Create a vocabulary containing unique words from the dataset
vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
            
vocabulary = list(set(vocabulary))
len(vocabulary)

7783

# The Final Training Set

In [10]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [11]:
# Transform word_counts_per_sms to a DataFrame
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head(3)

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# combined dataframes training_set and word_counts
training_set_cleaned = pd.concat([training_set, word_counts], axis=1)
training_set_cleaned.head(3)

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Calculating Constants First
Now that we're done with data cleaning and have a training set, we can begin creating the spam filter. To classify new messages, the Naive Bayes algorithm will need to answer these two probability questions.

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:
<img class="math math-display" alt="$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$" src="https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&amp;mode=display">
<img class="math math-display" alt="$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$" src="https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&amp;mode=display">

- α = Laplace smoothing
- NSpam is equal to the number of words in all the spam messages
- NHam is equal to the number of words in all the non-spam messages
- NVocabulary is equal to the number of unique words in all messages

In [13]:
# isolate spam and ham messages
spam_messages = training_set_cleaned[training_set_cleaned['Label'] == 'spam']
ham_messages = training_set_cleaned[training_set_cleaned['Label'] == 'ham']

# spam and ham probabilities
p_spam = len(spam_messages) / len(training_set_cleaned)
p_ham = len(ham_messages) / len(training_set_cleaned)

# n_spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# n_ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# n_vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

# Calculating Parameters
Now that the constant terms calculated above, we can move on with calculating the parameters. Each parameter will be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:
<img class="math math-display" alt="$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$" src="https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&amp;mode=display">
<img class="math math-display" alt="$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$" src="https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&amp;mode=display">

In [14]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum() # spam already defined in cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum() # ham already defined in cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

# Classifying A New Message
Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:
- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [15]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [16]:
classify('Congratulations you win! click on the link below to claim your prize.')

P(Spam|message): 1.3264525817289475e-30
P(Ham|message): 3.0217162650873457e-37
Label: Spam


In [17]:
classify("Sorry for the late response I was busy at work.")

P(Spam|message): 8.458766341414401e-35
P(Ham|message): 4.831524923388401e-28
Label: Ham


# Measuring the Spam Filters Accuracy
It's time to test how well the spam filter does on the test set.
The `classify()` function will output a label for every message in the test set, which will be compared with the actual label given by a human.

(`classify()` function will be edited, the labels will be returned instead of printing them.)

In [18]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that the function returns labels instead of printing them, it can be used to create a new column in the test set.

In [19]:
test_set['Predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,Predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Testing the accuracy of the spam filter

In [20]:
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['Predicted']:
        correct += 1
        
accuracy = correct / total
print('Correct:', correct)
print('Incorrect:', total-correct)
print('Accuracy:', accuracy)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


Out of 1,114 messages in the test set only 14 message were incorrectly labeled. Overall goal was to get an accuracy of at least 80%. The spam filter had an accuracy of about 98.74% which is very good.